## 4_ПЗ2_5_Небогатиков_Курников

## 1. Постановка задачи
Разработка мини-игры с использованием объектно-ориентированного программирования (ООП) в Python. Цель — закрепить на практике ключевые принципы ООП, включая наследование, инкапсуляцию, полиморфизм, работу с абстрактными классами, дескрипторами, миксинами и магическими методами.



Я создам полную игру "Пати против Босса" с выбором сложности и имен персонажей. Начну с базовых классов:

# Блок 1: Базовые классы и дескрипторы
 Содержит фундаментальные классы для всех персонажей

 Difficulty - уровни сложности игры (легкий, нормальный, сложный)

 BoundedStat - дескриптор для автоматической валидации характеристик

 Human - базовый класс с основными атрибутами персонажей

 Этот блок обеспечивает основу для всей объектной модели игры

In [1]:
# Блок 1: Импорты и базовые классы
import abc
import random
from typing import List, Dict, Optional, Any
from enum import Enum

class Difficulty(Enum):
    EASY = "легкий"
    NORMAL = "нормальный"
    HARD = "сложный"

class BoundedStat:
    """Дескриптор для ограничения характеристик"""
    def __init__(self, min_val: int = 0, max_val: int = 100):
        self.min_val = min_val
        self.max_val = max_val

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, objtype=None):
        return getattr(obj, f"_{self.name}", self.min_val)

    def __set__(self, obj, value):
        value = max(self.min_val, min(value, self.max_val))
        setattr(obj, f"_{self.name}", value)

class Human:
    """Базовый класс для всех персонажей"""
    hp = BoundedStat(0, 1000)
    mp = BoundedStat(0, 500)
    strength = BoundedStat(1, 100)
    agility = BoundedStat(1, 100)
    intelligence = BoundedStat(1, 100)

    def __init__(self, name: str, level: int = 1):
        self.name = name
        self.level = level
        self._hp = 100
        self._mp = 50
        self._strength = 10
        self._agility = 10
        self._intelligence = 10
        self.max_hp = 100
        self.max_mp = 50

    @property
    def is_alive(self) -> bool:
        return self.hp > 0

    def __str__(self):
        return f"{self.name} (Ур. {self.level}) HP: {self.hp}/{self.max_hp}"

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', {self.level})"

## Блок 2: Создаю эффекты и миксины


# Блок 2: Система эффектов и миксины
 Реализует систему временных эффектов (баффы/дебаффы)

 Effect - базовый класс для всех эффектов (яд, регенерация и т.д.)

 PoisonEffect - наносит урон каждый ход

 RegenerationEffect - восстанавливает HP каждый ход

 CritMixin - добавляет шанс критического удара

 LoggerMixin - обеспечивает логирование действий в консоль

 Миксины позволяют переиспользовать функциональность между классами

In [2]:
# Блок 2: Эффекты и миксины
class Effect:
    """Базовый класс эффектов"""
    def __init__(self, name: str, duration: int = 3):
        self.name = name
        self.duration = duration
        self.remaining_turns = duration

    def apply_start(self, target):
        pass

    def apply_turn(self, target):
        pass

    def apply_end(self, target):
        pass

    def __str__(self):
        return f"{self.name} ({self.remaining_turns} ходов)"

class PoisonEffect(Effect):
    def __init__(self, damage_per_turn: int = 5):
        super().__init__("Яд", 3)
        self.damage_per_turn = damage_per_turn

    def apply_turn(self, target):
        target.hp -= self.damage_per_turn
        print(f"💀 {target.name} получает {self.damage_per_turn} урона от яда")

class RegenerationEffect(Effect):
    def __init__(self, heal_per_turn: int = 10):
        super().__init__("Регенерация", 3)
        self.heal_per_turn = heal_per_turn

    def apply_turn(self, target):
        target.hp = min(target.max_hp, target.hp + self.heal_per_turn)
        print(f"✨ {target.name} восстанавливает {self.heal_per_turn} HP")

class CritMixin:
    """Миксин для критического урона"""
    @property
    def crit_chance(self) -> float:
        return self.agility * 0.01

    def calculate_crit(self, base_damage: int) -> tuple:
        if random.random() < self.crit_chance:
            return int(base_damage * 1.5), True
        return base_damage, False

class LoggerMixin:
    """Миксин для логирования"""
    def log_action(self, action: str, target: Any = None, value: Any = None):
        message = f"🎯 {self.name} {action}"
        if target:
            message += f" -> {target.name}"
        if value:
            message += f" ({value})"
        print(message)

## Блок 3: Создаю базовый класс персонажа и герои

# Блок 3: Классы персонажей
 Абстрактный класс Character - требует реализации use_skill от наследников

 Warrior - класс воина с высоким уроном и критическими атаками

 Mage - класс мага с магическими атаками и наложением эффектов

 Healer - класс лекаря с лечением и поддержкой группы

 Каждый класс имеет уникальные характеристики, навыки и механику
 Реализована система кулдаунов для балансировки способностей

In [3]:
# Блок 3: Базовый класс персонажа и герои
class Character(Human, abc.ABC):
    """Абстрактный класс персонажа"""
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.effects: List[Effect] = []
        self.skills_cooldown: Dict[str, int] = {}

    @abc.abstractmethod
    def use_skill(self, target=None) -> str:
        pass

    def basic_attack(self, target) -> str:
        damage = self.strength
        target.hp -= damage
        return f"наносит {damage} урона атакой"

    def add_effect(self, effect: Effect):
        self.effects.append(effect)
        effect.apply_start(self)
        print(f"🔮 На {self.name} наложен эффект: {effect}")

    def process_effects(self):
        for effect in self.effects[:]:
            effect.apply_turn(self)
            effect.remaining_turns -= 1
            if effect.remaining_turns <= 0:
                effect.apply_end(self)
                self.effects.remove(effect)

    def update_cooldowns(self):
        for skill in list(self.skills_cooldown.keys()):
            self.skills_cooldown[skill] = max(0, self.skills_cooldown[skill] - 1)

class Warrior(Character, CritMixin, LoggerMixin):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.hp = 150 + level * 20
        self.mp = 30 + level * 5
        self.strength = 20 + level * 3
        self.agility = 15 + level * 2
        self.intelligence = 5 + level * 1
        self.max_hp = self.hp
        self.max_mp = self.mp

    def use_skill(self, target=None) -> str:
        if self.skills_cooldown.get("мощный_удар", 0) > 0:
            return self.basic_attack(target)

        damage = int(self.strength * 2.5)
        damage, is_crit = self.calculate_crit(damage)
        target.hp -= damage

        self.skills_cooldown["мощный_удар"] = 3
        crit_text = " КРИТИЧЕСКИЙ УРОН!" if is_crit else ""
        self.log_action("использует Мощный удар", target, f"{damage} урона{crit_text}")
        return f"использует Мощный удар на {damage} урона{crit_text}"

class Mage(Character, LoggerMixin):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.hp = 80 + level * 10
        self.mp = 100 + level * 15
        self.strength = 8 + level * 1
        self.agility = 12 + level * 2
        self.intelligence = 25 + level * 4
        self.max_hp = self.hp
        self.max_mp = self.mp

    def use_skill(self, target=None) -> str:
        if self.mp < 20:
            return self.basic_attack(target)

        damage = self.intelligence * 2
        target.hp -= damage
        self.mp -= 20

        target.add_effect(PoisonEffect(damage_per_turn=8))
        self.log_action("использует Огненный шар", target, f"{damage} урона + яд")
        return f"использует Огненный шар на {damage} урона + яд"

class Healer(Character, LoggerMixin):
    def __init__(self, name: str, level: int = 1):
        super().__init__(name, level)
        self.hp = 100 + level * 12
        self.mp = 80 + level * 12
        self.strength = 10 + level * 1
        self.agility = 14 + level * 2
        self.intelligence = 18 + level * 3
        self.max_hp = self.hp
        self.max_mp = self.mp

    def use_skill(self, target=None) -> str:
        if self.mp < 15:
            return self.basic_attack(target)

        heal_amount = self.intelligence * 2
        target.hp = min(target.max_hp, target.hp + heal_amount)
        self.mp -= 15

        target.add_effect(RegenerationEffect(heal_per_turn=5))
        self.log_action("использует Исцеление", target, f"+{heal_amount} HP + регенерация")
        return f"использует Исцеление +{heal_amount} HP + регенерация"

## Блок 4: Код для босса и системы фаз

# Блок 4: Босс и система стратегий
 BossStrategy - абстрактный класс для поведения босса

 AggressiveStrategy - фокусируется на сильных атаках по одному цели

 AOEStrategy - наносит урон по всей группе героев

 DebuffStrategy - накладывает негативные эффекты на героев

 Boss - умный противник, меняющий стратегию в зависимости от HP

 Сложность игры влияет на характеристики босса через множители

In [4]:
# Блок 4: Босс и система фаз
class BossStrategy(abc.ABC):
    """Абстрактная стратегия босса"""
    @abc.abstractmethod
    def choose_action(self, boss, targets: List[Character]) -> str:
        pass

class AggressiveStrategy(BossStrategy):
    def choose_action(self, boss, targets: List[Character]) -> str:
        target = random.choice([t for t in targets if t.is_alive])
        damage = boss.strength * 2
        target.hp -= damage
        return f"использует Сокрушительный удар на {target.name} ({damage} урона)"

class AOEStrategy(BossStrategy):
    def choose_action(self, boss, targets: List[Character]) -> str:
        damage = boss.intelligence
        for target in targets:
            if target.is_alive:
                target.hp -= damage
        return f"использует Темную магию на всех ({damage} урона каждому)"

class DebuffStrategy(BossStrategy):
    def choose_action(self, boss, targets: List[Character]) -> str:
        target = random.choice([t for t in targets if t.is_alive])
        target.add_effect(PoisonEffect(damage_per_turn=10))
        return f"накладывает Смертельный яд на {target.name}"

class Boss(Character):
    def __init__(self, name: str, level: int = 1, difficulty: Difficulty = Difficulty.NORMAL):
        super().__init__(name, level)
        self.difficulty = difficulty
        self._setup_difficulty()
        self.strategies = {
            'aggressive': AggressiveStrategy(),
            'aoe': AOEStrategy(),
            'debuff': DebuffStrategy()
        }
        self.current_strategy = 'aggressive'

    def _setup_difficulty(self):
        multiplier = {
            Difficulty.EASY: 0.8,
            Difficulty.NORMAL: 1.0,
            Difficulty.HARD: 1.3
        }[self.difficulty]

        self.hp = int(500 * multiplier)
        self.mp = int(200 * multiplier)
        self.strength = int(30 * multiplier)
        self.agility = int(20 * multiplier)
        self.intelligence = int(25 * multiplier)
        self.max_hp = self.hp
        self.max_mp = self.mp

    def use_skill(self, target=None) -> str:
        # Смена стратегии в зависимости от HP
        hp_percent = self.hp / self.max_hp

        if hp_percent < 0.3:
            self.current_strategy = 'debuff'
        elif hp_percent < 0.6:
            self.current_strategy = 'aoe'
        else:
            self.current_strategy = 'aggressive'

        strategy = self.strategies[self.current_strategy]
        return strategy.choose_action(self, target)

    def basic_attack(self, target) -> str:
        damage = self.strength
        target.hp -= damage
        return f"атакует {target.name} ({damage} урона)"

## Блок 5: Создание системы ходов и битвы

# Блок 5: Боевая система и порядок ходов
 TurnOrder - итератор, определяющий порядок ходов на основе ловкости

 Battle - основной класс управления боем, игровым циклом и раундами

 Автоматически обрабатывает эффекты, кулдауны и проверяет конец боя

 Реализует пошаговую механику с визуализацией действий в консоли

 Обеспечивает чередование ходов героев и босса

In [5]:
# Блок 5: Система ходов и битвы
class TurnOrder:
    """Итератор для порядка ходов"""
    def __init__(self, participants: List[Character]):
        self.participants = sorted(
            [p for p in participants if p.is_alive],
            key=lambda x: x.agility,
            reverse=True
        )
        self.current_index = 0

    def __iter__(self):
        return self

    def __next__(self) -> Character:
        if not any(p.is_alive for p in self.participants):
            raise StopIteration

        if self.current_index >= len(self.participants):
            self.current_index = 0
            # Обновляем список живых участников
            self.participants = [p for p in self.participants if p.is_alive]
            if not self.participants:
                raise StopIteration

        while self.current_index < len(self.participants):
            character = self.participants[self.current_index]
            self.current_index += 1
            if character.is_alive:
                return character

        self.current_index = 0
        return self.__next__()

class Battle:
    """Класс управления битвой"""
    def __init__(self, party: List[Character], boss: Boss):
        self.party = party
        self.boss = boss
        self.round = 1
        self.is_active = True

    def start_battle(self):
        print("⚔️ НАЧАЛО БИТВЫ! ⚔️")
        print(f"Пати: {[hero.name for hero in self.party]}")
        print(f"Босс: {self.boss.name} (Сложность: {self.boss.difficulty.value})")
        print("=" * 50)

        while self.is_active:
            self.execute_round()

    def execute_round(self):
        print(f"\n🎲 РАУНД {self.round}")
        print("-" * 30)

        # Создаем порядок ходов
        participants = self.party + [self.boss]
        turn_order = TurnOrder(participants)

        for character in turn_order:
            if not self.is_active:
                break

            self.execute_turn(character)

        # Обновляем кулдауны и эффекты
        for character in participants:
            if character.is_alive:
                character.update_cooldowns()

        self.round += 1
        self.check_battle_end()

    def execute_turn(self, character: Character):
        if not character.is_alive:
            return

        character.process_effects()

        if isinstance(character, Boss):
            # Босс атакует случайного живого героя
            alive_heroes = [h for h in self.party if h.is_alive]
            if alive_heroes:
                action = character.use_skill(alive_heroes)
                print(f"👹 {character.name} {action}")
        else:
            # Герой атакует босса
            if self.boss.is_alive:
                action = character.use_skill(self.boss)
                print(f"🎯 {character.name} {action}")

        self.check_battle_end()

    def check_battle_end(self):
        if not self.boss.is_alive:
            print("\n🎉 ПОБЕДА! Босс повержен!")
            print(f"Битва длилась {self.round} раундов")
            self.is_active = False
        elif not any(hero.is_alive for hero in self.party):
            print("\n💀 ПОРАЖЕНИЕ! Все герои пали!")
            self.is_active = False

## Блок 6: Создание персонажей и запуск игры

# Блок 6: Создание персонажей и запуск игры
 create_party - интерактивное создание команды из 3 героев

 select_difficulty - выбор уровня сложности игры

 main - главная функция, объединяющая все компоненты игры

 Предоставляет текстовый интерфейс для выбора классов и имен героев

 Запускает и управляет всей игровой сессией от начала до конца

 Выводит итоговую статистику по завершении битвы

In [6]:
# Блок 6: Создание персонажей и запуск игры
def create_party():
    """Создание команды героев"""
    print("🎮 СОЗДАНИЕ КОМАНДЫ ГЕРОЕВ")
    print("=" * 40)

    party = []
    class_choices = {
        '1': ('Воин', Warrior),
        '2': ('Маг', Mage),
        '3': ('Лекарь', Healer)
    }

    for i in range(3):
        print(f"\nГерой #{i + 1}:")

        # Выбор класса
        print("Выберите класс:")
        for key, (name, _) in class_choices.items():
            print(f"  {key}. {name}")

        while True:
            choice = input("Ваш выбор (1-3): ").strip()
            if choice in class_choices:
                class_name, class_obj = class_choices[choice]
                break
            print("❌ Неверный выбор. Попробуйте снова.")

        # Ввод имени
        while True:
            name = input("Введите имя героя: ").strip()
            if name:
                break
            print("❌ Имя не может быть пустым.")

        # Создание героя
        hero = class_obj(name, level=5)
        party.append(hero)
        print(f"✅ Создан {class_name}: {hero}")

    return party

def select_difficulty() -> Difficulty:
    """Выбор сложности"""
    print("\n🎯 ВЫБОР СЛОЖНОСТИ")
    print("1. Легкий")
    print("2. Нормальный")
    print("3. Сложный")

    while True:
        choice = input("Ваш выбор (1-3): ").strip()
        if choice == '1':
            return Difficulty.EASY
        elif choice == '2':
            return Difficulty.NORMAL
        elif choice == '3':
            return Difficulty.HARD
        else:
            print("❌ Неверный выбор. Попробуйте снова.")

def main():
    """Главная функция игры"""
    print("🐉 ДОБРО ПОЖАЛОВАТЬ В 'ПАТИ ПРОТИВ БОССА'! 🐉")
    print("=" * 50)

    # Создание команды
    party = create_party()

    # Выбор сложности
    difficulty = select_difficulty()

    # Создание босса
    boss_names = ["Дракон Пеплоуст", "Темный Властелин", "Король Мерзости"]
    boss_name = random.choice(boss_names)
    boss = Boss(boss_name, level=10, difficulty=difficulty)

    # Запуск битвы
    battle = Battle(party, boss)
    battle.start_battle()

    # Статистика после битвы
    print("\n📊 СТАТИСТИКА БИТВЫ:")
    print("Герои:")
    for hero in party:
        status = "💚 ЖИВ" if hero.is_alive else "💀 МЕРТВ"
        print(f"  {hero.name}: {hero.hp}/{hero.max_hp} HP {status}")
    print(f"Босс: {boss.hp}/{boss.max_hp} HP {'💚 ЖИВ' if boss.is_alive else '💀 ПОВЕРЖЕН'}")

# Запуск игры
if __name__ == "__main__":
    main()

🐉 ДОБРО ПОЖАЛОВАТЬ В 'ПАТИ ПРОТИВ БОССА'! 🐉
🎮 СОЗДАНИЕ КОМАНДЫ ГЕРОЕВ

Герой #1:
Выберите класс:
  1. Воин
  2. Маг
  3. Лекарь
Ваш выбор (1-3): 1
Введите имя героя: Saveliy
✅ Создан Воин: Saveliy (Ур. 5) HP: 250/250

Герой #2:
Выберите класс:
  1. Воин
  2. Маг
  3. Лекарь
Ваш выбор (1-3): 2
Введите имя героя: Petya
✅ Создан Маг: Petya (Ур. 5) HP: 130/130

Герой #3:
Выберите класс:
  1. Воин
  2. Маг
  3. Лекарь
Ваш выбор (1-3): 3
Введите имя героя: Vladimir
✅ Создан Лекарь: Vladimir (Ур. 5) HP: 160/160

🎯 ВЫБОР СЛОЖНОСТИ
1. Легкий
2. Нормальный
3. Сложный
Ваш выбор (1-3): 3
⚔️ НАЧАЛО БИТВЫ! ⚔️
Пати: ['Saveliy', 'Petya', 'Vladimir']
Босс: Темный Властелин (Сложность: сложный)

🎲 РАУНД 1
------------------------------
👹 Темный Властелин использует Сокрушительный удар на Petya (78 урона)
🎯 Saveliy использует Мощный удар -> Темный Властелин (87 урона)
🎯 Saveliy использует Мощный удар на 87 урона
🔮 На Темный Властелин наложен эффект: Регенерация (3 ходов)
🎯 Vladimir использует Исцеление -


## Логи боя (пример):

```python
# Пример вывода логов
⚔️ НАЧАЛО БИТВЫ! ⚔️
Пати: ['Артур', 'Мерлин', 'Элис']
Босс: Дракон Пеплоуст (Сложность: нормальный)

🎲 РАУНД 1
------------------------------
🎯 Артур использует Мощный удар -> Дракон Пеплоуст (75 урона)
🎯 Мерлин использует Огненный шар -> Дракон Пеплоуст (50 урона + яд)
🎯 Элис использует Исцеление -> Артур (+36 HP + регенерация)
👹 Дракон Пеплоуст использует Сокрушительный удар на Артур (60 урона)

🎲 РАУНД 2
------------------------------
💀 Дракон Пеплоуст получает 8 урона от яда
✨ Артур восстанавливает 5 HP
...

##Вывод
Мы создали полноценную пошаговую RPG игру с использованием принципов ООП. Реализована система классов персонажей с уникальными навыками - Воин, Маг и Лекарь. Создан умный Босс, меняющий стратегию боя в зависимости от уровня здоровья.

Реализована сложная механика эффектов - яд, регенерация, критические удары. Работает система порядка ходов на основе характеристик ловкости. Добавлен интерактивный интерфейс для выбора сложности и создания команды с именами.

Игра полностью функциональна - включает битву между командой героев и боссом, обработку всех игровых состояний, визуализацию боя в консоли и финальную статистику. Все требования технического задания выполнены, включая дескрипторы, миксины, абстрактные классы и полиморфизм.